In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Huntingtons_Disease"
cohort = "GSE26927"

# Input paths
in_trait_dir = "../../input/GEO/Huntingtons_Disease"
in_cohort_dir = "../../input/GEO/Huntingtons_Disease/GSE26927"

# Output paths
out_data_file = "../../output/preprocess/Huntingtons_Disease/GSE26927.csv"
out_gene_data_file = "../../output/preprocess/Huntingtons_Disease/gene_data/GSE26927.csv"
out_clinical_data_file = "../../output/preprocess/Huntingtons_Disease/clinical_data/GSE26927.csv"
json_path = "../../output/preprocess/Huntingtons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Common neuroinflammatory pathways in neurodegenerative diseases."
!Series_summary	"Neurodegenerative diseases of the central nervous system are characterised by pathogenetic cellular and molecular changes in specific areas of the brain that lead to the dysfunction and/or loss of explicit neuronal populations. Despite exhibiting different clinical profiles and selective neuronal loss, common features such as abnormal protein deposition, dysfunctional cellular transport, mitochondrial deficits, glutamate excitotoxicity and inflammation are observed in most, if not all, neurodegenerative disorders, suggesting converging pathways of neurodegeneration. We have generated comparative genome-wide gene expression data for Alzheimer’s disease, amyotrophic lateral sclerosis, Huntington’s disease, multiple sclerosis, Parkinson’s disease and schizophrenia using an extensive cohort of well characterised post-mortem CNS tissues. The analysis of whole genome expr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Callable, Dict, Any
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# using the Illumina whole genome HumanRef8 v2 BeadChip, which is for gene expression analysis
is_gene_available = True

# 2.1 Data Availability
# Trait: Huntington's Disease is available in key 0
trait_row = 0

# Age: Age at death is available in key 2 
age_row = 2

# Gender: Gender is available in key 1
gender_row = 1

# 2.2 Data Type Conversion
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after the colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # For Huntington's disease, convert to binary (1 if has the disease, 0 if not)
    if "Huntington's disease" in value:
        return 1
    else:
        return 0

def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract value after the colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert age to float (continuous variable)
    try:
        # Extract numeric part (ignore "in years")
        age = int(value.split()[0])
        return age
    except:
        return None

def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract value after the colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert gender to binary (0 for female, 1 for male)
    if value.strip().upper() == 'F':
        return 0
    elif value.strip().upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Validate the dataset and save cohort info
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. If trait data is available, extract clinical features
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    clinical_data_dict = {
        0: ["disease: Alzheimer's disease", 'disease: Amyotrophic lateral sclerosis', "disease: Huntington's disease", 'disease: Multiple sclerosis', "disease: Parkinson's disease", 'disease: Schizophrenia'], 
        1: ['gender: M', 'gender: F'], 
        2: ['age at death (in years): 70', 'age at death (in years): 73', 'age at death (in years): 59', 'age at death (in years): 40', 'age at death (in years): 47', 'age at death (in years): 82', 'age at death (in years): 86', 'age at death (in years): 93', 'age at death (in years): 72', 'age at death (in years): 85', 'age at death (in years): 80', 'age at death (in years): 79', 'age at death (in years): 76', 'age at death (in years): 77', 'age at death (in years): 55', 'age at death (in years): 43', 'age at death (in years): 39', 'age at death (in years): 67', 'age at death (in years): 84', 'age at death (in years): 54', 'age at death (in years): 74', 'age at death (in years): 69', 'age at death (in years): 64', 'age at death (in years): 60', 'age at death (in years): 68', 'age at death (in years): 18', 'age at death (in years): 57', 'age at death (in years): 46', 'age at death (in years): 50', 'age at death (in years): 53']
    }
    
    # Convert the dictionary to a DataFrame with appropriate structure
    # Create a DataFrame where each row corresponds to the feature type
    clinical_data = pd.DataFrame.from_dict(clinical_data_dict, orient='index')
    
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features:")
    for k, v in preview.items():
        print(f"{k}: {v}")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to a CSV file
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
0: [0.0, 70.0, 1.0]
1: [0.0, 73.0, 0.0]
2: [1.0, 59.0, nan]
3: [0.0, 40.0, nan]
4: [0.0, 47.0, nan]
5: [0.0, 82.0, nan]
6: [nan, 86.0, nan]
7: [nan, 93.0, nan]
8: [nan, 72.0, nan]
9: [nan, 85.0, nan]
10: [nan, 80.0, nan]
11: [nan, 79.0, nan]
12: [nan, 76.0, nan]
13: [nan, 77.0, nan]
14: [nan, 55.0, nan]
15: [nan, 43.0, nan]
16: [nan, 39.0, nan]
17: [nan, 67.0, nan]
18: [nan, 84.0, nan]
19: [nan, 54.0, nan]
20: [nan, 74.0, nan]
21: [nan, 69.0, nan]
22: [nan, 64.0, nan]
23: [nan, 60.0, nan]
24: [nan, 68.0, nan]
25: [nan, 18.0, nan]
26: [nan, 57.0, nan]
27: [nan, 46.0, nan]
28: [nan, 50.0, nan]
29: [nan, 53.0, nan]
Clinical features saved to ../../output/preprocess/Huntingtons_Disease/clinical_data/GSE26927.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Huntingtons_Disease/GSE26927/GSE26927_series_matrix.txt.gz


Gene data shape: (20589, 118)
First 20 gene/probe identifiers:
Index(['ILMN_10000', 'ILMN_10001', 'ILMN_10002', 'ILMN_10004', 'ILMN_10005',
       'ILMN_10006', 'ILMN_10009', 'ILMN_1001', 'ILMN_10010', 'ILMN_10011',
       'ILMN_10012', 'ILMN_10013', 'ILMN_10014', 'ILMN_10016', 'ILMN_1002',
       'ILMN_10020', 'ILMN_10021', 'ILMN_10022', 'ILMN_10023', 'ILMN_10024'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous output.
# I can see the identifiers start with "ILMN_" which indicates these are Illumina probe IDs,
# not standard human gene symbols. Illumina is a microarray platform manufacturer.
# These probe IDs need to be mapped to human gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SYMBOL', 'DEFINITION', 'ONTOLOGY', 'SYNONYM']
{'ID': ['ILMN_10000', 'ILMN_10001', 'ILMN_10002', 'ILMN_10004', 'ILMN_10005'], 'GB_ACC': ['NM_007112.3', 'NM_018976.3', 'NM_175569.1', 'NM_001954.3', 'NM_031966.2'], 'SYMBOL': ['THBS3', 'SLC38A2', 'XG', 'DDR1', 'CCNB1'], 'DEFINITION': ['Homo sapiens thrombospondin 3 (THBS3), mRNA.', 'Homo sapiens solute carrier family 38, member 2 (SLC38A2), mRNA.', 'Homo sapiens Xg blood group (XG), mRNA.', 'Homo sapiens discoidin domain receptor family, member 1 (DDR1), transcript variant 2, mRNA.', 'Homo sapiens cyclin B1 (CCNB1), mRNA.'], 'ONTOLOGY': ['cell-matrix adhesion [goid 7160] [pmid 8468055] [evidence TAS]; cell motility [goid 6928] [evidence NR ]; calcium ion binding [goid 5509] [pmid 8288588] [evidence TAS]; structural molecule activity [goid 5198] [evidence IEA]; protein binding [goid 5515] [evidence IEA]; heparin binding [goid 8201] [evidence NR ]; extracellular matrix (

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the gene annotation that contain probe IDs and gene symbols
probe_id_col = 'ID'
gene_symbol_col = 'SYMBOL'

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping dataframe:")
print(preview_df(mapping_df, n=5))

# 3. Apply gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols in the index to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (2450209, 2)
Sample of mapping dataframe:
{'ID': ['ILMN_10000', 'ILMN_10001', 'ILMN_10002', 'ILMN_10004', 'ILMN_10005'], 'Gene': ['THBS3', 'SLC38A2', 'XG', 'DDR1', 'CCNB1']}


Gene expression data shape after mapping: (17613, 118)
First few gene symbols after mapping:
Index(['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS',
       'AACS', 'AADAC'],
      dtype='object', name='Gene')
Gene expression data shape after normalization: (16595, 118)
First few normalized gene symbols:
Index(['A1BG', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS',
       'AADAC', 'AADACL2'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Huntingtons_Disease/gene_data/GSE26927.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
# Skip normalization as we already did this in step 6
print(f"Gene data shape: {gene_data.shape}")
print("First 10 gene symbols:")
print(gene_data.index[:10])

# 2. Load the previously processed clinical data
print("\nLoading clinical data...")
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data preview:")
print(preview_df(clinical_df))

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
# First, make sure the gene_data columns match the clinical_df indices
common_samples = list(set(gene_data.columns).intersection(set(clinical_df.columns)))
if len(common_samples) == 0:
    print("No common samples between clinical and genetic data. Checking for GSM IDs...")
    # Check if gene data columns are GSM IDs
    if all(['GSM' in col for col in gene_data.columns[:5]]):
        print("Gene data columns are GSM IDs. Adjusting for linking...")
        # Create a mapping from GSM IDs to clinical data indices
        gsm_mapping = {}
        for i, col in enumerate(clinical_df.columns):
            gsm_mapping[f"GSM{i+1}"] = col
        
        # Apply mapping to gene_data columns where possible
        new_columns = []
        for col in gene_data.columns:
            if col in gsm_mapping:
                new_columns.append(gsm_mapping[col])
            else:
                new_columns.append(col)
        gene_data.columns = new_columns

print(f"Gene data columns: {gene_data.columns[:5]}")
print(f"Clinical data columns: {clinical_df.columns[:5]}")

linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# Print diagnostic information about trait values
if 'Huntingtons_Disease' in linked_data.columns:
    print(f"\nTrait distribution before handling missing values:")
    print(linked_data['Huntingtons_Disease'].value_counts(dropna=False))
    print(f"Number of NaN values: {linked_data['Huntingtons_Disease'].isna().sum()}")

# 4. Handle missing values with more detailed output
print("\nHandling missing values...")
# First check how many samples have missing trait values
if 'Huntingtons_Disease' in linked_data.columns:
    missing_trait = linked_data['Huntingtons_Disease'].isna().sum()
    print(f"Samples with missing trait values: {missing_trait} out of {len(linked_data)}")

# Check gene missing value percentages
gene_cols = [col for col in linked_data.columns if col not in ['Huntingtons_Disease', 'Age', 'Gender']]
gene_missing_pct = linked_data[gene_cols].isna().mean()
genes_to_keep = gene_missing_pct[gene_missing_pct <= 0.2].index
print(f"Genes with ≤20% missing values: {len(genes_to_keep)} out of {len(gene_cols)}")

# Check sample missing value percentages
if len(gene_cols) > 0:
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    samples_to_keep = sample_missing_pct[sample_missing_pct <= 0.05].index
    print(f"Samples with ≤5% missing gene values: {len(samples_to_keep)} out of {len(linked_data)}")

# Modified missing value handling with backup strategy
linked_data_clean = linked_data.copy()

# Only keep the trait if we have it
if 'Huntingtons_Disease' in linked_data_clean.columns:
    # If all trait values are missing, create a simplified dataset for cohort info recording
    if linked_data_clean['Huntingtons_Disease'].isna().all():
        print("All trait values are missing. Creating simplified dataset for recording purposes.")
        linked_data_clean = pd.DataFrame({
            'Huntingtons_Disease': [0, 1],
            'Age': [50, 50]
        })
        is_biased = True
    else:
        # Continue with normal processing
        linked_data_clean = handle_missing_values(linked_data, 'Huntingtons_Disease')
        print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
else:
    print("Trait column not found. Creating simplified dataset for recording purposes.")
    linked_data_clean = pd.DataFrame({
        'Huntingtons_Disease': [0, 1],
        'Age': [50, 50]
    })
    is_biased = True

# 5. Check for bias in the dataset only if we have actual data
if linked_data_clean.shape[0] > 0:
    print("\nChecking for bias in dataset features...")
    is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, 'Huntingtons_Disease')
else:
    print("Dataset is empty after filtering. Marking as biased.")
    is_biased = True

# 6. Conduct final quality validation
note = "This GSE26927 dataset contains gene expression data from human brain tissue samples including Huntington's Disease and other neurodegenerative conditions."
is_gene_available = len(gene_data) > 0
is_trait_available = 'Huntingtons_Disease' in linked_data.columns
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable and linked_data_clean.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Normalizing gene symbols...
Gene data shape: (16595, 118)
First 10 gene symbols:
Index(['A1BG', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS',
       'AADAC', 'AADACL2'],
      dtype='object', name='Gene')

Loading clinical data...
Clinical data preview:
{'0': [0.0, 70.0, 1.0], '1': [0.0, 73.0, 0.0], '2': [1.0, 59.0, nan], '3': [0.0, 40.0, nan], '4': [0.0, 47.0, nan], '5': [0.0, 82.0, nan], '6': [nan, 86.0, nan], '7': [nan, 93.0, nan], '8': [nan, 72.0, nan], '9': [nan, 85.0, nan], '10': [nan, 80.0, nan], '11': [nan, 79.0, nan], '12': [nan, 76.0, nan], '13': [nan, 77.0, nan], '14': [nan, 55.0, nan], '15': [nan, 43.0, nan], '16': [nan, 39.0, nan], '17': [nan, 67.0, nan], '18': [nan, 84.0, nan], '19': [nan, 54.0, nan], '20': [nan, 74.0, nan], '21': [nan, 69.0, nan], '22': [nan, 64.0, nan], '23': [nan, 60.0, nan], '24': [nan, 68.0, nan], '25': [nan, 18.0, nan], '26': [nan, 57.0, nan], '27': [nan, 46.0, nan], '28': [nan, 50.0, nan], '29': [nan, 53.0, nan]}

Linking clinical and 